In [3]:
import numpy as np
import pandas as pd

from __future__ import division

### Description
Say we have an N sided die and list, B, of length N filled with 0's and 1's.

<br> If we roll a value on the die and the index of B (1-based indexing, here) associated with that value is 0, then whatever value we rolled, we get that number of dollars. E.g. B = [0,0,0,1] and we roll 2 => we get $2.

<br> On the other hand, if we roll a value, m, such that the mth index of B equals 1, then we lose all of our money and the game ends. E.g. B = [0,1,0,1] and we roll 4 => We lose the game and all our money.

<br>
Our goal then is to determine, given our current state in the game, whether or not we should reroll.

<br>
Below we have a class that takes in a given B and tells us what rolls it could take were it to follow the optimal Markov Decision Process, the outcomes of those rolling sequences, and the expected value of this input B.

In [125]:
class NSidedDice:
    def __init__(self, B):
        '''
        INPUT
            B: tuple of 0's and 1's. 1 indices represent numbers s.t. the game ends.
        OUTPUT
            N: Number of sides on die.
            fail_prob: Probability of failing die roll.
            state: The amount of money we've made from the game.
            win_indices: which numbers, if we roll them, will win us money
            path_probs: Probability of following each path
            path_values: Reward quantity for a given path
            path_rolls: list of lists of rolls composing each path.
        '''
        self.N = len(B)
        self.B = B
        self.fail_prob = np.sum(B) / len(B)
        self.state = 0
        self.win_indices = [n for n in range(self.N) if self.B[n] == 0]
        self.path_probs = []
        self.path_values = []
        self.current_prob = 1 
        self.path_rolls = []
        self.current_rolls = []
    
    def calculate_expected_values(self):
        '''
        calculates expected value of rolling dice.
        '''
        expected_gains = np.sum([(1 / self.N) * (val + 1) for val in self.win_indices])
        expected_loss = self.fail_prob * self.state
        
        self.expected_value = expected_gains - expected_loss
        

    def run_bellman(self):
        '''
        Runs simulation of all possible outcomes to calculate Bellman Equation
        '''
        for i in range(self.N):
            self.current_prob *= 1 / self.N
            self.current_rolls.append(i + 1)
            # case where we rolled a winning number
            if i in self.win_indices:
                self.state += (i + 1)
                self.calculate_expected_values()
                if self.expected_value <= 0:
                    self.path_probs.append(self.current_prob)
                    self.path_values.append(self.state)
                    self.path_rolls.append(self.current_rolls)
                    # case where we pop up a level in our tree
                    if i == self.N - 1 and len(self.current_rolls) > 1:
                        self.current_prob *= self.N**2
                        self.state -= self.current_rolls[-2]
                        self.state -= self.current_rolls[-1]
                        self.current_rolls = self.current_rolls[:-2]
                    else:
                        self.current_prob *= self.N
                        self.state -= self.current_rolls[-1]
                        self.current_rolls = self.current_rolls[:-1]

                # if expected value is positive, we roll again
                else:
                    if len(self.current_rolls) < 100:
                        self.run_bellman()
            # case where we rolled a number in B
            else:
                self.path_probs.append(self.current_prob)
                self.path_values.append(0)
                self.path_rolls.append(self.current_rolls)
                if i == self.N - 1 and len(self.current_rolls) > 1:
                    self.current_prob *= self.N**2
                    self.state -= self.current_rolls[-2]
                    self.current_rolls = self.current_rolls[:-2]
                else:
                    self.current_prob *= self.N
                    self.current_rolls = self.current_rolls[:-1]
        
    def get_expected_value(self):
        self.run_bellman()
        self.expected_value = np.sum(
            [prob * val for prob, val in zip(self.path_probs, self.path_values)]
        )

In [141]:
dice = NSidedDice([1,0,0])
dice.get_expected_value()

In [142]:
dice.expected_value

2.2592592592592591

In [143]:
dice.path_rolls

[[1], [2, 1], [2, 2, 1], [2, 2, 2], [2, 2, 3], [2, 3], [3, 1], [3, 2], [3, 3]]

In [144]:
dice.path_values

[0, 0, 0, 6, 7, 5, 0, 5, 6]

In [145]:
dice.path_probs

[0.3333333333333333,
 0.1111111111111111,
 0.037037037037037035,
 0.037037037037037035,
 0.037037037037037035,
 0.1111111111111111,
 0.1111111111111111,
 0.1111111111111111,
 0.1111111111111111]